# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
import sys
import importlib

# List of packages to check
packages = ["transformers", "torch", "datasets", "numpy", "pandas"]

print(f"🧠 Python version: {sys.version}\n")

for pkg in packages:
    try:
        module = importlib.import_module(pkg)
        version = getattr(module, "__version__", "Version info not found")
        print(f"✅ {pkg} version: {version}")
    except ImportError:
        print(f"❌ {pkg} is not installed.")

🧠 Python version: 3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]

✅ transformers version: 4.36.0
✅ torch version: 2.0.1
✅ datasets version: 2.18.0
✅ numpy version: 1.26.4
✅ pandas version: 2.2.1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [ ]:
# Saving the model
model.save("/tmp/your_model_name")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.